In [3]:
from langchain.llms.openai import OpenAI #llm 모델 선택 (davinci 는 비싸)
from langchain.chat_models import ChatOpenAI #chat 모델 선택, #ChatAnthropic

#llm = OpenAI()
chat = ChatOpenAI() # ChatAnthropic() 모델 바로 바꿀수 있어
# chat모델은 chat에 특화
# a = llm.predict("How many planets are there")
b = chat.predict("How many planets are there") 
b

'In our solar system, there are eight planets: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. However, there are potentially thousands of exoplanets in other solar systems beyond our own.'

In [6]:
from langchain.chat_models import ChatOpenAI #chat 모델 선택, #ChatAnthropic
from langchain.prompts import PromptTemplate, ChatPromptTemplate #ChatPromptTemplate : 은 아래 'messages'로 부터 프롬프트를 만들고 / PromptTemplate : 은 string으로부터 만듬

chat = ChatOpenAI(temperature=0.1) 

template = PromptTemplate.from_template("What is the distance between {place_a} and {place_b}? Also, what is your name?")
prompt = template.format(place_a='Mexico', place_b='Thailand')

chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles). My name is Assistant.'

In [7]:
# # predict message
# from langchain.schema import HumanMessage, AIMessage, SystemMessage #system message : 우리가 llm설정들을 제공하기 위한 message
# messages = [
#     SystemMessage(content = "You are a geography expert. And you only reply in {language}"),
#     AIMessage(content = "안녕하세요, 저는 대한민국 대동여지도를 만든 {name} 입니다."),
#     HumanMessage(content = "What is the distance between {place_a} and {place_b}? Also, what is your name?")
# ]

# chat.predict_messages(messages)

In [8]:
# tempplate 생성
template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}"),
    ("ai", "안녕하세요, 저는 대한민국 대동여지도를 만든 {name} 입니다."),
    ("human", "What is the distance between {place_a} and {place_b}? Also, what is your name?")
])

prompt = template.format_messages(
    language = "Greek",
    name = "Socrates",
    place_a = "Mexico",
    place_b = "Thailand"
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.')

In [9]:
# Output Parser필요한 이유는 llm의 응답을 변형해야할 때가 있기 때문 (리스트로 변환하던지)
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    
    def parse(self,text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
p = CommaOutputParser()
p.parse("Hello,how, are,you")

['Hello', 'how', 'are', 'you']

In [16]:
template = ChatPromptTemplate.from_messages(
    [("system","You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}. Do NOT reply with anything else.",),
     ("human","{question}"),]
)
prompt = template.format_messages(max_items=10, question="What are the colors?")
result = chat.predict_messages(prompt)

p = CommaOutputParser()
p.parse(result.content)

['Red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [17]:
# Better way.
# 코드를 다시 정리 해보자 (여기서 부터 코드 다시 돌려봐)
chat = ChatOpenAI(temperature=0.1) 

class CommaOutputParser(BaseOutputParser):
    
    def parse(self,text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

template = ChatPromptTemplate.from_messages(
    [("system","You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}. Do NOT reply with anything else.",),
     ("human","{question}"),]
)

# Chain을 만들거야
chain = template | chat | CommaOutputParser()
chain.invoke({  #chain 실행, dictionary형태로
    "max_items" : 5,
    "question" : "What are the Pokemons"}) 

['Pikachu', 'Charmander', 'Bulbasaur', 'Squirtle', 'Jigglypuff']

In [23]:
# 2 chain = chain +chain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1, # 낮을수록 창의적
    streaming=True,  # 실시간으로 글자 보여줘 
    callbacks=[StreamingStdOutCallbackHandler()]) # console에 보이는 문자를 다 print해줘 (다양한 llm에서 일어나는 event를 감지하는 방법)

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any types of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat

In [24]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modifiy the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}"),
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain # "recipe" context object를 체인에 넣어줘 (아래 코드 안쓰고, recipe 값에 결과 저장 후, 다음 체인으로 전달)
final_chain.invoke({ 
    "cuisine":"indian"
})

#"recipe": chef_chain 이것을 함으로서 자동으로 veg_chain.invoke() 가 돌아가는 것임

# # 즉, chef_chain의 결과 {recipe}를 veg_chain의 input 으로..
# veg_chain.invoke({
#     "recipe":"chatmodel" #이런식으로
# })

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic and delicious recipe for Chicken Tikka Masala. Here's how you can make it at home:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 teaspoons ground cumin
- 2 teaspoons paprika
- 1 teaspoon ground turmeric
- 1 teaspoon garam masala
- 1 teaspoon ground coriander
- 1 teaspoon chili powder (adjust to taste)
- 3 cloves garlic, minced
- 1 tablespoon grated fresh ginger
- Salt and pepper to taste
- 2 tablespoons vegetable oil
- 1 onion, finely chopped
- 1 can (14 oz) tomato sauce
- 1 cup heavy cream
- Fresh cilantro, chopped (for garnish)
- Cooked rice or naan bread (to serve)

Instructions:
1. In a bowl, combine the yogurt, lemon juice, cumin, paprika, turmeric, garam masala, coriander, chili powder, garlic, ginger, salt, and pepper. Add the chicken pieces and mix until well coated. Cover and marina

AIMessageChunk(content="As a vegetarian chef, I can help you make a vegetarian version of Chicken Tikka Masala by replacing the chicken with a suitable alternative. In this case, you can use paneer, a type of Indian cottage cheese that is commonly used in vegetarian dishes as a substitute for meat. Here's how you can modify the recipe:\n\nIngredients:\n- 1 lb paneer, cut into bite-sized pieces\n- 1 cup plain yogurt\n- 2 tablespoons lemon juice\n- 2 teaspoons ground cumin\n- 2 teaspoons paprika\n- 1 teaspoon ground turmeric\n- 1 teaspoon garam masala\n- 1 teaspoon ground coriander\n- 1 teaspoon chili powder (adjust to taste)\n- 3 cloves garlic, minced\n- 1 tablespoon grated fresh ginger\n- Salt and pepper to taste\n- 2 tablespoons vegetable oil\n- 1 onion, finely chopped\n- 1 can (14 oz) tomato sauce\n- 1 cup heavy cream\n- Fresh cilantro, chopped (for garnish)\n- Cooked rice or naan bread (to serve)\n\nInstructions:\n1. Follow the same marinating process as the original recipe, but sub

In [ ]:
# # 모듈
# LangChain은 다음과 같은 주요 모듈을 위한 확장 가능한 표준 인터페이스 및 외부 통합 기능을 제공합니다:

# - Model I/O
# 언어 모델과의 인터페이스

# - Retrieval
# 애플리케이션별 데이터를 사용한 인터페이스

# - Agents
# 높은 수준의 지침이 주어지면 체인이 어떤 도구를 사용할지 선택하도록 합니다

# - Chains
# 일반적인 빌딩 블록 구성

# - Memory
# 체인 실행 간 지속적인 응용 프로그램 상태

# - Callbacks
# 모든 체인의 중간 단계 기록 및 스트리밍

# few shot 
# you are giving examples to model how to answer

In [26]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatMessagePromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1, # 낮을수록 창의적
    streaming=True,  # 실시간으로 글자 보여줘 
    callbacks=[StreamingStdOutCallbackHandler()]) # console에 보이는 문자를 다 print해줘 (다양한 llm에서 일어나는 event를 감지하는 방법)

t = PromptTemplate.from_template("What is the capital of {country}")
t.format(country='France')

'What is the capital of France'

In [27]:
# #이 형식도 똑같아 (langchain document way)
# from langchain.chat_models import ChatOpenAI
# from langchain.prompts import ChatMessagePromptTemplate
# from langchain.prompts.few_shot import FewShotPromptTemplate
# from langchain.callbacks import StreamingStdOutCallbackHandler

# chat = ChatOpenAI(
#     temperature=0.1, # 낮을수록 창의적
#     streaming=True,  # 실시간으로 글자 보여줘 
#     callbacks=[StreamingStdOutCallbackHandler()]) # console에 보이는 문자를 다 print해줘 (다양한 llm에서 일어나는 event를 감지하는 방법)

# t = PromptTemplate(
#     template = "What is the capital of {country}",
#     input_variables = ['country'])

# t.format(country='France')

'What is the capital of France'

In [35]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatMessagePromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate, 
from langchain.callbacks import StreamingStdOutCallbackHandler


chat = ChatOpenAI(
    temperature=0.1, # 낮을수록 창의적
    streaming=True,  # 실시간으로 글자 보여줘 
    callbacks=[StreamingStdOutCallbackHandler()]) # console에 보이는 문자를 다 print해줘 (다양한 llm에서 일어나는 event를 감지하는 방법)

#example
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]
#format examples
# example_template = """Human: {question} AI : {answer}"""
# example_prompt = PromptTemplate.from_template(example_template)
# or just 
example_prompt = PromptTemplate.from_template("Human: {question}\n AI : {answer}")


prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
    suffix = "Human: What do you know about {country}?", #example 다 넣고, question of user
    input_variables = ["country"]
)

#prompt.format(country="Germany")

chain = prompt | chat
chain.invoke({"country":"Germany"})

AI: 
        I know the following:
        Capital: Berlin
        Language: German
        Food: Bratwurst and Sauerkraut
        Currency: Euro

AIMessageChunk(content='AI: \n        I know the following:\n        Capital: Berlin\n        Language: German\n        Food: Bratwurst and Sauerkraut\n        Currency: Euro')

In [40]:
# chatprompt templete 사용해보기
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatMessagePromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1, # 낮을수록 창의적
    streaming=True,  # 실시간으로 글자 보여줘 
    callbacks=[StreamingStdOutCallbackHandler()]) # console에 보이는 문자를 다 print해줘 (다양한 llm에서 일어나는 event를 감지하는 방법)

#example
examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece", # 참고
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_prompt =  ChatPromptTemplate.from_messages(
    [
    ("human", "What do you know about {country}?"), #위 참고 가 country
    ("ai", "{answer}")
])


example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
)


final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert"),
    example_prompt,
    ("human", "What do you know about {country}?")
])

chain = final_prompt | chat
chain.invoke({"country":"Germany"})


        I know this:
        Capital: Berlin
        Language: German
        Food: Bratwurst and Sauerkraut
        Currency: Euro
        

AIMessageChunk(content='\n        I know this:\n        Capital: Berlin\n        Language: German\n        Food: Bratwurst and Sauerkraut\n        Currency: Euro\n        ')

In [4]:
#  Dynamic example selector
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatMessagePromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector # 예제의 양을 얼마나 되는지 확인하고, 세팅값에 따라 알맞은 prompt예제 골라줄 것임.

        

chat = ChatOpenAI(
    temperature=0.1, # 낮을수록 창의적
    streaming=True,  # 실시간으로 글자 보여줘 
    callbacks=[StreamingStdOutCallbackHandler()]) # console에 보이는 문자를 다 print해줘 (다양한 llm에서 일어나는 event를 감지하는 방법)

#example
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = PromptTemplate.from_template("Human: {question}\n AI : {answer}")

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=80,
)

prompt = FewShotPromptTemplate( 
    example_prompt = example_prompt,
    example_selector=example_selector, #추가 (이제는 전체 예제를 사용하는게 아니라 selector를 거친 예제들만)
    suffix = "Human: What do you know about {country}?", 
    input_variables = ["country"]
)

prompt.format(country='Brazil')

NameError: name 'PromptTemplate' is not defined

In [ ]:
#  Dynamic example selector
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatMessagePromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector # 예제의 양을 얼마나 되는지 확인하고, 세팅값에 따라 알맞은 prompt예제 골라줄 것임.
from langchain.prompts.example_selector.base import BaseExampleSelector


        

chat = ChatOpenAI(
    temperature=0.1, # 낮을수록 창의적
    streaming=True,  # 실시간으로 글자 보여줘 
    callbacks=[StreamingStdOutCallbackHandler()]) # console에 보이는 문자를 다 print해줘 (다양한 llm에서 일어나는 event를 감지하는 방법)

#example
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

class RandomExampleSelector(BaseExampleSelector): #랜던으로 예제 선택하는 
    
    def __init__(self, examples):
        self.examples = examples
    
    def add_example(self, example): 
        self.examples.append(example)
    
    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)] #리스트로해서 넣어야하니까 
    

example_prompt = PromptTemplate.from_template("Human: {question}\n AI : {answer}")

example_selector = RandomExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=50,
)

prompt = FewShotPromptTemplate( 
    example_prompt = example_prompt,
    example_selector=example_selector, #추가 (이제는 전체 예제를 사용하는게 아니라 selector를 거친 예제들만)
    suffix = "Human: What do you know about {country}?", 
    input_variables = ["country"]
)

prompt.format(country='Brazil')